<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-OSSP1-Debugger-4/blob/autoencoder/Training_K_Face_model_with_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set the Library

In [ ]:
# 라이브러리 설정
import numpy as np
import cupy
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import cv2
import gc
import glob
from tensorflow.keras.layers import Reshape
from keras.models import load_model
import keras

# 랜덤 시드 고정
SEED=2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

 # Load the NPZ file containing K-FACE Images (For Pre-trained Model)

In [ ]:
load_y=np.load('../Upscaling/Dataset_output/NON-WEAR_dataset.npz') 
Y_1 =load_y['arr_0']
load_y.close()


#Test Data
Y_test = Y_1[-1000:] 
Y_1 = Y_1[:-1000]


del load_y
print(Y_1.shape)
gc.collect()

(21400, 800, 800, 3)


8

#Using Multi GPU

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:2","/gpu:3","/gpu:4","/gpu:5","/gpu:6","/gpu:7"])

8 Physical GPUs, 8 Logical GPUs
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


# Set the Model_checkpoint & Train the Model

In [ ]:
#모델 체크포인트 설정
checkpoint_path = './check/checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                monitor='val_acc',
                                                verbose=1)

In [ ]:
with strategy.scope():
    history = ae_model.fit(Y_1,
                       Y_1,
                       batch_size=128, #한번에 학습할 데이터 Batch_size
                       epochs=300,      #에포크 설정
                       verbose=1,      #학습과정 시각화     
                       callbacks=[checkpoint],  #모델 체크포인트 저장
                       validation_split=0.2)         

#Save the model
ae_model.save('./check/model2.h5')

Test the Model (Pre-trained Model)

In [ ]:
ae_images = ae_model.predict(Y_test)  #Pre-trained Model 평가용
ae_images.shape

Print the Predict Result (Pre-trained Model)

In [ ]:
#이미지 출력
num = 10 
plt.figure(figsize=(20,8))
for i in range(10):
    # 원본 이미지
    ax = plt.subplot(2, num, i+1)
    plt.imshow(Y_test[i*10])
    plt.title("Original %s" % str(i))
    plt.axis('off')
    
    # 복원 이미지
    ax = plt.subplot(2, num, i+num+1)
    plt.imshow(ae_images[i*10])
    plt.title("Auto-encoded %s" % str(i))
    plt.axis('off')
    
plt.show()

# ================ Transfer Learning (For User) ================

#Load the Pre-trained Model

In [ ]:
with strategy.scope():
    ae_model = load_model('./check/models/model2.h5')

#Load the Dataset for Transfer Learning (Video Frame set + Input Image)

In [ ]:
#팀장 데이터 불러오기 (Pre-Trained Model에 추가학습용) -> 동영상 프레임 추출 사진
minsu = glob.glob('./TransferLearningImages/videoimage3/*.png')
test_minsu = []
for _ in range(0,100):
  img = cv2.imread(minsu[_])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  test_minsu.append(img.astype("float32")/255.0)
  

#합성사진 추가
testpath = '800x800.png'
img = cv2.imread(testpath)
img=(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
test_minsu.append(img.astype("float32")/255.0)
test_minsu.append(img.astype("float32")/255.0)
test_minsu.append(img.astype("float32")/255.0)
test_minsu.append(img.astype("float32")/255.0)
test_minsu.append(img.astype("float32")/255.0)

val_minsu=[]
val_minsu.append(test_minsu[-1])
test_minsu = np.array(test_minsu)
val_minsu = np.array(val_minsu)
np.random.shuffle(test_minsu)

print(test_minsu.shape)

# Training the Model

In [ ]:
ae_model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='mean_squared_error', metrics=['val_acc']) 
ae_model.summary()

In [ ]:
#모델 체크포인트 설정
checkpoint_path = 'checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                monitor='val_acc',
                                                verbose=1)

In [ ]:
history = ae_model.fit(test_minsu,
                       test_minsu,
                       batch_size=10, 
                       epochs=30,    
                       verbose=1,
                       validation_data=(val_minsu, val_minsu),                                                        
                       callbacks=[checkpoint])  #모델 체크포인트 저장
                      
#Save the MoDEL
ae_model.save('./check/transfer.h5')

# Generate & Show the Output

In [ ]:
trans_imgs = ae_model.predict(val_minsu)

In [ ]:
plt.imshow(trans_img[0])
plt.show()

# Save the Output Images

In [ ]:
cv2.imwrite('output.jpg', cv2.cvtColor(trans_img[0]*255, cv2.COLOR_BGR2RGB))